In [1]:
import numpy as np
import pandas as pd
import json
import os
import datetime
import math
import re
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import random
random.seed(1)
from PIL import Image
import sys
sys.path.insert(1, '../../measures/intricacy/')
from calculate_intricacy import *
sys.path.insert(1, '../../measures/local spatial complexity/')
from calculate_local_spatial_complexity import *
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
sns.set_theme()
sns.set(font_scale=1.5, font="serif")
pd.set_option('display.max_colwidth', 10000)

In [2]:
def decrypt(encrypted, c1, c2):
    enckeys = "abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789()!*;:@&+$/#[].~-,"
    reversedict = {}
    total_bin = int(math.pow(2, segsize));
    reversedict = {
      '!': "0", '#': "9", '$': "7", '&': "5", '(': "14", ')': "15", '*': "1", '+': "6", ',': "15", '-': "14", '.': "12", '/': "8", '0': "4",
      '1': "5", '2': "6", '3': "7", '4': "8", '5': "9", '6': "10", '7': "11", '8': "12", '9': "13", ':': "3", ';': "2", '@': "4", '[': "10",
      ']': "11", 'a': "0", 'A': "31", 'b': "1", 'B': "30", 'c': "2", 'C': "29", 'd': "3", 'D': "28", 'e': "4", 'E': "27", 'f': "5", 'F': "26", 'g': "6", 'G': "25",
      'h': "7", 'H': "24", 'I': "23", 'i': "8", 'J': "22", 'j': "9", 'K': "21", 'k': "10", 'L': "20", 'l': "11", 'M': "19", 'm': "12", 'N': "18", 'n': "13",
      'O': "17", 'o': "14", 'P': "16", 'p': "15", 'q': "16", 'Q': "26", 'r': "17", 'R': "27", 's': "18", 'S': "28", 't': "19", 'T': "29", 'u': "20",
      'U': "30", 'v': "21", 'V': "31", 'W': "0", 'w': "22", 'X': "1", 'x': "23", 'Y': "2", 'y': "24", 'Z': "3", 'z': "25", '~': "13"
    }
    patt = []
    allpatt = re.findall(c1, encrypted)
    pc = len(allpatt)
    for i in range(pc):
        grid = []
        rowenc = re.findall(c2, allpatt[i])
        assert(len(rowenc) == grid_size)
        for j in range(len(rowenc)):
            row = []
            rowinfo = rowenc[j]
            assert(len(rowinfo) == numseg)
            for k in range(len(rowinfo)):
                decval = reversedict[rowinfo[k]];
                binval = "{0:b}".format(int(decval)).zfill(segsize)
                assert(len(binval) == segsize)
                for l in range(len(binval)):
                    row.append(int(binval[l]));
            grid.append(row)
        patt.append(grid)
    return patt

In [3]:
def get_15_mins_from(timestamp):
    hr, min, sec = [int(i) for i in timestamp.split('-')]
    time = datetime.time(hr, min, sec)
    time_plus_15 = datetime.datetime.combine(datetime.date.today(), time) + datetime.timedelta(minutes = 15)
    tstmp = str(time_plus_15.hour).zfill(2) + "-" + str(time_plus_15.minute).zfill(2) + "-" + str(time_plus_15.second).zfill(2)
    return time_plus_15.time()

def gettimebetweenseconds(a, b):
    time_start, time_end = a.split('-'), b.split('-')
    hr_start, hr_end, min_start, min_end, sec_start, sec_end = int(time_start[0]), int(time_end[0]), int(time_start[1]), int(time_end[1]), int(time_start[2]), int(time_end[2])
    a = datetime.time(hr_start, min_start, sec_start)
    b = datetime.time(hr_end, min_end, sec_end)
    if b < a:
        return 0
    tdiff = datetime.datetime.combine(datetime.date.today(), b) - datetime.datetime.combine(datetime.date.today(), a)
    return tdiff.seconds

def gettimebetween(a, b):
    time_start, time_end = a.split('-'), b.split('-')
    year_start, year_end, month_start, month_end, day_start, day_end, hr_start, hr_end, min_start, min_end, sec_start, sec_end = int(time_start[0]), int(time_end[0]), int(time_start[1]), int(time_end[1]), int(time_start[2]), int(time_end[2]), int(time_start[3]), int(time_end[3]), int(time_start[4]), int(time_end[4]), int(time_start[5]), int(time_end[5])
    a = datetime.datetime(year_start, month_start, day_start, hr_start, min_start, sec_start)
    b = datetime.datetime(year_end, month_end, day_end, hr_end, min_end, sec_end)
    tdiff = b - a
    return tdiff.seconds / 60 #minutes

In [4]:
def get_demog_data(d):
    gender = d['gender']
    try:
        age = int(d['age'])
    except:
        age = -1
    nationality = d['nationality']
    dominant_hand = d['dominant_hand']
    device = d['device']
    mouse = d['mouse']
    art_answers = d['art_answers']
    return gender, age, nationality, dominant_hand, device, mouse, art_answers

In [5]:
def get_behav_data(d, filedataset):

    start_grid_time = d['time_per_grid'][0] # first click on "Next/Start"
    end_grid_reqd_time = get_15_mins_from(start_grid_time)
    end_grid_time = d['time_per_grid'][-1]
    num = 0
    for tsind in range(len(d['time_per_grid'])):
        ts = d["time_per_grid"][tsind]
        hr, min, sec = [int(i) for i in ts.split('-')]
        time = datetime.time(hr, min, sec)
        if time <= end_grid_reqd_time:
            num += 1
    num += 1 # add last grid
    tstmp = d['time_per_grid'][num - 1]

    time_per_grid = [gettimebetweenseconds(i, j) for i, j in zip(d['time_per_grid'][:num-1], d['time_per_grid'][1:num])]
    num_grids_visited = num - 1 # d['num_grids_visited'] - 1 # Remove first as example grid
    
    # for data2, num should be equal to num_grids_visited
    if filedataset == 2:
        assert num == d['num_grids_visited']
    
    mins = gettimebetweenseconds(tstmp, end_grid_time)/60

    # for data2, there should be no time between end_grid_time and num time
    if filedataset == 2:
        assert mins == 0

    time_taken = gettimebetween(d['start_time'], d['end_time']) - mins
    
    clicks_per_grid = d['clicks_per_grid'][1:num]
    # avg_clicks_per_grid = d['average_clicks_per_grid']
    avg_clicks_per_grid = np.mean(clicks_per_grid)
    var_clicks_per_grid = np.var(clicks_per_grid)
    
    click_positions = d['click_positions'][1:num]

    click_timestamps = d['click_times'][1:num]
    click_times = []
    for ind, t in enumerate(click_timestamps):
        t.insert(0, d['time_per_grid'][ind])
        click_times.append([gettimebetweenseconds(i, j) for i, j in zip(t[:-1], t[1:])])
    
    strategy = d['strategy']
    gridsmostopened = d['gridmostopened']
    gridsleastopened = d['gridsleastopened']
    try:
        moveon = d['moveon']
    except:
        moveon = "NA"
    graytiles = d['graytiles']
    ocd = d['ocd']
    fun = d['fun']
    opencomments = d['opencomments']

    return time_taken, num_grids_visited, avg_clicks_per_grid, var_clicks_per_grid, clicks_per_grid, click_positions, time_per_grid, click_times, strategy, gridsmostopened, gridsleastopened, moveon, graytiles, ocd, fun, opencomments

In [6]:
def get_exper_data(d):
    exampleexplorationstimuli = decrypt(d['exampleexplorationPattern'], case1, case2)[0]
    order_of_ranges = d['order_of_ranges']
    order_within_ranges = d['order_within_ranges']
    return exampleexplorationstimuli, order_of_ranges, order_within_ranges

In [7]:
def get_pattern_order(oor, owr):
    stimuli = []
    for o in oor:
        for w in owr[o]:
            stimuli.append(ranges[o][w])
    return stimuli

In [8]:
def modify_grid(pattern, grid, pos):
    if pos[0] % 3 == 0:
        xpos = [pos[0], pos[0] + 1, pos[0] + 2]
    elif pos[0] % 3 == 1:
        xpos = [pos[0] - 1, pos[0], pos[0] + 1]
    elif pos[0] % 3 == 2:
        xpos = [pos[0] - 2, pos[0] - 1, pos[0]]
    if pos[1] % 3 == 0:
        ypos = [pos[1], pos[1] + 1, pos[1] + 2]
    elif pos[1] % 3 == 1:
        ypos = [pos[1] - 1, pos[1], pos[1] + 1]
    elif pos[1] % 3 == 2:
        ypos = [pos[1] - 2, pos[1] - 1, pos[1]]
    
    for xind in xpos:
        for yind in ypos:
            grid[xind, yind] = pattern[xind][yind]
    return grid

In [9]:
def calculate_measures(grid):
    LSC = calculate_local_spatial_complexity(grid, grid_size)
    intricacy = calculate_intricacy(grid, grid_size)
    return LSC, intricacy

In [10]:
def matrix_to_pattern(p):
    ratio = 22
    new_data = np.zeros(np.array(p.shape) * ratio)
    for j in range(p.shape[0]):
        for k in range(p.shape[1]):
            if p[j, k] == 2:
                new_data[j * ratio: (j+1) * ratio, k * ratio: (k+1) * ratio] = 2
            elif p[j, k] == 1:
                new_data[j * ratio: (j+1) * ratio, k * ratio: (k+1) * ratio] = 1
            elif p[j, k] == 0:
                new_data[j * ratio: (j+1) * ratio, k * ratio: (k+1) * ratio] = 0
    cmap = matplotlib.colors.ListedColormap([cmapmap[i] for i in np.unique(new_data)])
    plt.imshow(new_data, cmap=cmap)
    plt.axis('off')
    plt.show()


def pattern_to_str(filepath):
    ratio = 40
    img = Image.open(filepath).convert('L')	
    big = np.array(img)
    small = np.array([big[i, 0::ratio] for i in range(0,len(big),ratio)])
    small[small == 0] = 1
    small[small == 255] = 0
    return ''.join(map(str, small.flatten()))

In [11]:
segsize = 3
grid_size = 27
cmapmap = {0: "white", 1: "black", 2:"gray"}
numseg = grid_size/segsize

case1 = '.{1,243}'
case2 = '.{1,9}'
ranges = [
            decrypt("WaWaaW!W!a!aW!aWWaaW!aaWaWW!aaWW!WaaW!W!WWaa!a!WaWaa!a!!WaaaWWW!!a!a!aWaWW!WW!WWWa!W!!a!!W!a!WWWaWW!WaZc2a!!!aa3h3W!aWaah$3!!a!Wah3hW!a!a!X3eaW!W!aX$e!aaWWa!YW!!aW!W!!a!Waa!!aWa!WWaW!!!W!!W!!aaa!WWaa!W!Wa!WWa!!!!Waa!WaaWW!Wa!!a!a!!a!a!WaaWW!aWaWaaaW!!Wa!a!!aaaa:aWa!WWZ0:$W!WaWde*hWaaWWh0a3!WWaX$W!3!h33$e!!Wbh3$$eW!a*3303W!aa:0!W+WaWa:@!a+a!WX$@WWh+aWXb0aa$2!*$!a*h$h@:h!Wb$$h2Z3W!Xh33+dhW!XgZXgdh$$h@cX+Xh$$3WX!gb$$$ha:!!bh3$3aZW!hh$@d!!!a33$eW!aaWh33h0aaa!Wd$32!WgW!:+Zg!WheWZ@aaaaZ0$3h333hhh$W:!;!gWh2@$0$XhXd1+Z3$h+Zf0f*h3h01be:e$33XgXfXh333$0f3e$h3$hbhh23h3hhd31$3$$$3h&e33$3$$$*0$33$3h3Xf;3:Y2hc&h2;$hh;d$hhahYhWhh1::3hh+gffdh1$fh+1$g$$3$3d33g+$;3Zd3$Zdh$3+g$$1f2c:1&$hg2:a+ddh&3:bfeg3&1333ah3h123$$3h33:hX+:3+d@33h$h33$hh$33h$$h$h0aaa!aWWbf3$33h33&fWa!!aWa&fdh$hhh21fcWaaWac&&;hh3$$c&fY@:&g*c&&c1+hd&;1f;fh$3fc&fc3gf:hc11cg&h&Z;11;3$f33cf1;1fa&1Y11ch3&h3cf1c+fhfdY&fch+1Zh;f1Yf$hh1cffc&g3:fc1f;e:&2Xc&1;h3h3h;ff;WaW!!;1&:h3hhh+1&aWW!Wa!11hh$3$h$&0!!Waaa!*33hhh$$3$Ygg!0::a*0aWYb!a!3!!e@Wf&bY!;1!W1*a$2YWW0aa!!aac+2;c!@!hYW!@d!bYWaWW!WZXcY*W+&!XaWg:*a0!eZcW!a!c&b!XYac3aabb0&dWYXY@;+aa;bccgeac!@;aWW1e0g*!3e0!0!c0e:3a;11W2WW:!1W:;WYca0!a0*Z;cgbZ;a@*!abXa0fWWa0W0d!!!0cXa!0Z:&a1!@c!WWW0Y*a;!efc;!c;e*c1;ehXY@Y$b2W!Wdeh$$3h333h331gWW!df3Zhh3333$gc$2acad$Y$$h$1$33h3$+!&ad$h3$+b1eZhhc&b3!hefYY&b$&h0fc;fb2aZ@f;cfZ111+fcc12f!fd&;;2aa!Wad;c121W1:&;;1Z&1&g1Y;f*gW:e&Y;&*h&$0fccfXh!h01Y33+*fe:hh33ga1!Z33$hh3f3$3$c3g!Y!:3YZ$333$h$2$fgWWW:&$h$33hh33$hX+a!Wd0hW&0YX1hda;eZ&hYZ!cd@ed!b2:+;:&c@cab&0*1ch0YWd!23d;&*+*Z0hZWf!e*d!X&:Ya*1hWgbg$3h*Wh+a2@b21:;XdaaW$!*@1&c&@*Ze!XaYXcg+3aa$&gY+;:ac!c2cb;!*a!e2h3@W!aZZ2*;*0c;X1h&0fabb$ad1;@df0X1cfXeZW;gZ;+g!1g:@!!f:@12XfdWbc2XX2!;abW$$ce;0a:aX;$3a!@:Wg&c@0:e1c!", case1, case2),
            decrypt("a!W!WaW!aWa!Waaaa!aWaaWWaWWW!aaaW!a!WW!WWaaa!!a!W!Wa!a!a!WWa!!!WaaWWWaaa!!!!!!!a!!!!acaWWW!aW!YWW!!W!aWfaaWaWWWbYe!Waa!a+1ZaWWaW!*c0!WaW!aa1!aWa!!a!cW!WWaWa!Yaa!a!aWW!!!W!WW!aWaW!WWWWaaa!!aaaWaaWa!aa!aWWWa!aa!WWWaaW!a!WWWaWa!a!a!WaaWW!a!!W!!aW1bhh$h$e1c23$333Zc&d3333hg&;$33h$hhYZhh3$h$3g1h$$3h3$1h33hhh33$$333h$hh3h$$h$$$hh$$$$1h$$3$h33f$$$$3hhhW3hh$$33g1Z$hh$h3!a!$$$h$3+1:$h$h3h3!$$3hh3hhf3h3$$$$31$$33h$h$h$h$hh$h$$$h$33h333hh33fh3$$hh31Zh3$$3h$gYh$$$3h3c1Zhhhhh+f;+333h$:;1*$h3$h@&3hh$$$$$3fh3h3g$gbh33hh$3$13cd3@*a3$$hh3WWW:hhh3$aaW:hf$$g!WW:$3$$0aW!$33h*!!aX3h$;!a!aZ3f$W@a;!Z313X+W3a3:3hd$X31$h3333332$h&hd33h@:d$$Xhh$a*+1h!$$+WZh&fe33hahf33Xh&gWZh13&33eb*h33g3$gd3$1heZh$hh$$3!*h3h$hhhW*hhhh$:3$h33h3$*1$$e*22c*$33h3hh331$;3$33$h0W:!WaW!!h3a:hh+&&0adeW!W1&ghY*hY@W*1aXYbc$h101&YfcaW&gg&c&c$gf!WfYXa!!&h$1d;13c1e!c0Y&!cff:;XY1Zgf1Wb1;f!ab1:ea;&d$f1W:$cWaWX@$aW1ed$&+WZ+@WaWX;h@!fdf310@cg1!Y!bgec0fhYf&e*!!!bc@&fg3h3&Y2fe!!WaXWWf1$3$$@!2*eWa!adW*fe:Yg&W@0!Y:c2&3$3h*3+W$!Z3@:hh$fh33gh&bZ32@f331@;Y;*1h2ZhYcc$2Zh2fYY;&d$23dZ3gg$Z+YhZ&2$cdZ:g@hb:2+;Wb@$*0ac:3gZ;gdh+:W2:W2:W+$W3@hX3ah1$1g3dfh13a$@3XhW3d!gdW2dWg:h+Zc2dh2;W*@$X@aY:Z+@3XZ+g+;3Z&+$Y:g3d:322hZ$gf;cYfdh2dhYY;3g:3fec;;bfhh1*:$201$Z$$$1$$$gX$+!3W:$@;0:e$*2bcf+db;@gZ1YWdh;h+ac2X$e;XheZcd023:*2Yag$gf:hd!W$bac!ehW3fbfc1efh3f2Z!2d&$13;fh1Y$1*:*W3We+@Z!Z01b2Wg@:Wh1$!gb32+2WddZ$0dWh&3a+X:W:e1Xg!gXdb!hW@g01$;1$1;$&$&2:a2d&331*f;1@1ha$*WcWe$!a+$2&:$Za;:0+3dbgY+X$0Yb$0dc!:3Yh2!c1+:*c@2dfY0Z03b2bc@e&bWa&3e*&!2ZZf@XW+aX:ccW;X0a+1f:X+$$03W3:!ff:+:XX$;XXa0e+Xbe*gd+fedW@hYfb:+!0e@c2*YcW@a2dbd3ecd!X!;b&Y2*3g*b3e1Z2hb;fcc0g213*$e0fh1bb&*e22b;3001*+g:gZ2$eab!cY*X3W;$ge0e*Y1WY@1**+&$02h$11Z0@Z1fXha$@!!he:3+Z0d@h3;02@&X*2&*h3W*ZY1*d&2d0$32:fg+&", case1, case2),
            decrypt("3WWWWaW!heaaWa!a!X@!W!a!aW*aaWaa!a!WWaWa!aW!!!!aa!a!WWaa!Wa!W!W!!a!!aaWaWW!Wa!!aW!!a!W!a!aaa!WaWW!!!!!!XW!aWaWWWX!WWaa!W!!!!!WaWaae!!a!!WW!!!WW!aWaWWa!aWW!!!aWWWaa!WWW!a!aaWWaWWW!aWa!aaW!W!W!WW!a!Wa!aaW!WW!W!!!WaW!Wae!aWaWW!*@WWaa!W!X3Wa!WW!!3+!*3hh@!:@a:33$+W*W!3$3$h!W!Xhh$3$@a!:$h$$32!!33$$h$$!bhh$3h3heZ3h$h$3hg$hh$3h$3$3h$hh$$$3h3hhhh$$3h$3+aZ$$33$hgc:3h33h3+hdh$3$$$+YZhhh$33+adh33$$3h$$$$hh3$h33hh333$h$h$$3Z33hh33$+*$hhhh3$0!hh333hhW!:33$h3gWa*$$33h@WWW3h3$$Wae!Zh$$2aXga*$3$ead:3h$;hh$+f;W3h$Wcf+2WYaca::1gaa!WW:&$+aW!W!:heWa!aa!W*@!Waa!aWbeWWa*!a!XeaaW@aWWXga!b2aa!Zha!Y33W!3g!W1W&X@dcaXfh1aW;g!ad@baWdcaW$f32!c2!*:f:+!d$aWefbde3+a!db:0aZ0WbZX!!WXe!!a+a!!b0!a!aaWa*@a!aaa!a*3gaW!!a:3&+aWaa!d&2g!YWcadZ1Y!3$3a;1Zh3$;$$hg$hh33h3$h@aa!a!!!X&hh$hh3$f&!WaWWaWf&Zh$3$3g11YWW!!ac1f;$$$$$Yf&;@:W+XY&f;fYaY&;&1Y@eaXb;ff;$W1W$cffYgXh0ZYf&c@:f2XY1&ceb!@*;ffc@:&gXY&fYg*3eZc&fc$!&!hc1fce@aXX;&fc&c!Y1c11Y@da+Xcff;3333$;1&Y!W!!!;1&:3h$$hgffa!WaaaW1f3h3h$$$f@aWa!!aaX$$h3hh$3$aW!Wb3@Wace@@fh1bcaW!abh0!WY@aa*0Z&YWWW;f0*@aaf@W!W!bacebcY!!1ca!*WWaWXW+WeY!!c!a+aac@f;!cXWWaaaccWb;Wa$h$a;XWXWh$h!abX!W!W@c1+!@!*;1c!2&!a!Wed$+eXYbWW$32e0Zh;c0d2aYb@Wa&:a*XXecb@ZY0aYZ@X&dW2ecZab0:!ag2Waa:WYfWWeW*dY!aW@:&aaac01@!!b*caWb0a:eWW$1Xh&$e1$0!g!aW:W*ea+aWa:WbeaeaWaXWXW!hW$ahaa0!$aa!$!*:$2;hYdhg::eWh!b+geZbZ$+@2*eaaZ$gW!X@!&3@X&!*@aXhf10WXe;h$g:$c*a;$$2:3ca@Y33&1$;*eab3e*@!X@Wfh$h1a*0W!d32!ab@:b:32@+b::@a3!X2g:$+Y3;Z$+eW$Waa3!bWW3WhW3Wa@!@a!!bW*@a+!Wa:W*e!+WW!da*$fb$f30f$a*ff$1f0aaY&d3+&Ya!2Wd!+WZ!bb$$f3$e0d:*Z12@g2@2d*c02:b+eYg$ZY*dW&@YhYb&!g3!0$bWh:ee1aW!f*bdg+aWWdZ2h$aa!!a3h2g$!!!hZZ+X$!!!$@dgg$aaahdd3$!W!a!$3d+2WWWZZ2@efW!!fXXg$a@h*a3ZW1eYhc*1!2@Yg3dY*:@g:b;0+Z*ZZX:12e2g*b3hf3h00WgW:a+WZWac&Z$21;!!Xff$110a", case1, case2),
            decrypt("W!aWWaaWa!!!Wa!WWaWWa!aaa!a!aW!!!!a!WaaWWa!!!!!WW!Wa!!aaa!!W!W!!a!W!WWa!!!W@abWWWWaaWaa@aaaW*aWX0WWW!WWab0!!aa!Wb@a!aW!aa!a!WWW!WaX@!W!!!!a!aaaaaa!a1$@!!W!ae1$e!WaW*g!:@W!a!!WWaa!aW!a!aWW!aWa!W!WWa!WWaaa!a!W!a!a!aa!WWaW!!!aaa!a!a!Waaaa!WaWaWW!cf;1;fcf;1c&cf;&;1Y1Y1YfYfY&c&Y1c1c1cf;&Yf;1Yfc1Yfc1;&Yfc&Yfc1;fY&Y1Y1cfcfc&;1c&;1;1c&;f;&Y&c&YfY1;&;&c&Yfcf;&Y&Y1cfYY&;1*;fc&f;&;h;1Y1Y&Y&!&c1;fc1chc1;1c&Y1!fcfc&c1Y$;fY1Y&Yfa&;1c&Y&c3;&c&;1Y1a1;&Y&c&c3Y&c&c&c1a1YfY1;&;hY1c1Y&;1!f;fc&cf;$cfY133h3333$$hef$33&*3+$d$hh23d$2$3$33Z3f&h$$hh&f1Z3$h33gf+h$3$$$hZ&h$3$33$&3$$$3$hh$3h$Zh2$33h$3&c&3h$h332$dh$hh$hfYf3333$$31333h$3h1Y&h3$$h$2hZh$$3h3&;1$h$h$3d32h3h$33$$h3$$&h$hh3$h&+$3$h3h$:1Zhhh33g1f&3$$h3&fhg3hh3$Z$ghZ3hh+$:$efh33fX$$3$3$$$33$h3$$hhhh@aaaaWaa*f33$333311!!!WWWa1fZ3$hh3+ffYa!aaaY&f;33$$$;ff;@h2hbY11Y@f*hXY11;3h*:3Y&1;@@hhhYf&Y@$b@3c11cdX*@$Y&fY:f+:*;ff;d+cdX;ffc2+:03c&1;3f0e&cffcg:@h3Y11;+c3XXcffY3h**bc&fY$$hhhYf1YaaWWaY&&Zh3h$$2&faaW!WWa1f33h3hh3&eWWWWaWaXhh$$$33h3*hWaa!W$eYbe;Wc*@;0!3aYW$aX0b0XW0*0*e;a;aYa;X2@bda+@X::@!*f@!Xg*a;a$W;!0bbXWY!@e0!WW@W*a!aYdWWY!a+ca1@ahW*faa!+*&e:!aXaddW2ga@X0:dW2+*eXe2*&0dXe:W0aaWXaga&a0!Xa&!W*@!!W*0aYaY0aX;W;*e*@ab@Xea&*Wa!efW!!W!$aW!aef@bf@*1*eW0:a+Xa*;a;!YWYWYb0aaWa!*@WZ$ehb$+!!3$*&0$3a*32dW2Z3@Z$02;dbhgh3aZ!+!hhh2!bheW:3h0a:hg!b3h!ahf$Wa$2abh!$@aZ@@d+;:+b*Xghe$b3Z0::3!!ah++2X2Y&;d0:0&egW:bf*2X+;1cd@d:Z3a!ah22X2h@h*3Zee0d+YZg**+Wb$ah0!d$W!3&h!W33@WZ$+Wb3$+!X$0WZ$h3!Z!2!$3:h@2cdXhg*$g:a2d3@!h$*&@3haaZh@$X3gW;YWeXc!h0WYdac&*df02hX*W0;W*YYc1+:$Z;Z$2*2:db2;:+b1;a;W0WWWhWa:baWc:@12Xe+33Z0*:b1h@bb!Yh*Z@c@f00;;a*+h&1Zb$WfY&W:$&+*W::b*;Wea@f2$**g&Wb$2bb!Zc!Z;!YZ!aX0Y@cfYe;dYccYe!cY2YYc2W0Y@de2+3e*f;XaYW0+Wf@a+Z0b!:b$hf$db:deZW:1e::fe1Yf!Yhf3@+@cdY0X;Z!@", case1, case2),
            decrypt("$;3ehX3ch1hh2hZ331$hh33hhhhZ$h33hh$+3$$333$h3d$$$$hhhghhh$h33h$$$3h333$h3h$h3h3h$$h$$$hh33:33h$$33+bhh$3hh3@$$$h3$$h33h$3$333hh$$hh3$$h*h$$h$$30:3$3333$2$$h$$$h3$$3$$$h$$$3$3$3$3$$3h3$h3$hhZhh3h3hhghh3h33h3$:h$$h3hh2h333$h$$$13$ghZ$$&3c$@hb3;h!aW!aWaa!!WWW!aWWa!aWaW!aW!WW!WWWWWW!W!!caa!W!a!WhWWa!!aa!;a!!WWWaW&aW!a!a!bce!aWaWa;1Y!a!WW!&c&aa!!a*Y1ceaWWXYf;&Ye!a:1c&c&g!!*;1;&Y0!W!*;fc@aW!WW1cfaW!!!W;fY!a!aa!bc@W!W!!aWf!aW!!aWac!!!WaWaW3aWWWW!WaYW!WW!W!aWWWW!!aa!WW!aa!W!!aWaWWW!!WW!!!WW!!Waaaaaa!!WWaWWZWWWWaWW!d!aa!WaW!!WWWaa!aWa!a!WW!a!WW!aagXeWa!a!Wgb0!Z!gaaa!W!ZW+WaWaW!W!+W!a!WWWa+Waa!aW!W&h!!!!!!!h3!aaW!a!!!W!aWa!W!aa!W!a!aW!!W!W!+!!gWa!W!2a!2!W!!d2W!2aW:a:gaa2WWZ!W:aaWaadWW:!a!WWdaaW!a!!Waa!aaad+bea!!$ad+*0W!Wh!aWaaa!Whhh$$h333&$$h1$3313&$3a$h13ha1+aZfW$g!:0WXga:$!$fZc@a33&f$$32b$hdh1h2hY$$$b$ehgh3$g:3:dh:3h@3Ze3:f3$ad$h3$!$2!bch&cad@ad$3h$WX+a3Z03Z0:3!Z3d:3W33eX$e$2X$3gdf3g$:33hh33hh$33d2f:c123$&2WaW:&$$WhW!a$!3+Wdea*+a:hWf+!Z1!h31h3!3$fh133$fh33&h$h$h$333!a3$2Wa:eaXh0X1cbaab3g!&2$0aXZ2h2@3aad$g1;WWWW!$1+c!3ea*+!0$cge!XYgcYY&0WZc:@ag!WWW3;!aa2a*@Zh2;cdW!d:hbgX:!!h32!e3&WX3h3+1$*fW$hh3a*;c*$h33f0Ya*:33$$h!0!d$gZe@Ze!X1h$fdg!!W:aheaaaa!Y:2eWaaW!X;ba!WWbfW1!!a0!ba+c!Wa@a!2XY!W!aaa!Wfa*!W!W!!!!*WW2d3e:h2Xh2&ff$$$1f&$2:X302d$3e2$$3Z*3fb&fff1@1+;X3;3ec:2ge:Y2b:dX0:da2+*0d:ce$Xc2g;3*@hb0hc+f+ZcgZfZ33+Z!gd$$3g*@h*0Z$hf1g3Zf1$3+*0$b0d33$gZagZhh+12:;g:1:c$*eh*@hY:Z;@$*Y+gX@d:ag+beg+0:;2XZdgYX3c$ec:f*fff&1@&h0+hh3ZX$$g:*$egdh&11$$$1&f:h@d$gbh+XWfa+ge:@X3WY*e*1f;Xh2W2e@X!bb!afh3@cW!3b@ac!+$+a0@f0@dcdcgWc*$:0g2aef;3ecZ@W0d;Y3:W1@YX3@Zg*31:c2b*2aaXY@!ZXYXc0fXaWh;0dg*&ed@3aaWXfZ33hccdc*gabf@cc*$f!@*c;20*0c:d;3+b!0$:*;+@XW:e;*;dWa:!ZXW*$0c&ca&;3db1;c@&$bh&e*+*3bW;*f&df!hd:3@hc$c020a;&0*", case1, case2),
            decrypt("0aWWa!!ab!WW!!!a!WaW!a!WaaW!!!!!WaW!aaW!aW!!W!W!WaW!WaaWa!a!!Wa!Waaa!a!aaaWaWWa!WaaWaaWa!aW!aaW!aaa!aaW!W!!!!Wa!aa!WWaaWa!!aWW!!a!aaa!!!aa!W!Waa!a!!aW!Waaa!a!W!a!!!aa!WaW!W!WWW!aa!WWa!W!WW!!WW!WW!!!WW!!aaa!aW!!a!!W!!aWWaW!aW!WWWWWWWaa0!aaWWWWb;0cf;&cXcfa&caYf!1c&;1;fY&;&cfcfY&Y&!&Y1c1;fabcfY&Y&Y@c1Y&c1c1c&Y0;1cX;1;fa1;1!f;fY1Y&Y1c1;1c1cf;&c1;&c&c1Y1a&YfYfcfa1cf;f;f;1Wfc1Y&Yfa1;&Y&c1;&;&YfcfY1;&c1YfYf;&c&W1YfW&;&Y0cfYbY1Y1;1Y1Y&YbY&YfY&;ea&cf;1;&a1;fYfcf;fY1c1c1cfc&!1Y!;fafYe;f;&Yb;hhhh$3333h$h3h3hh3$h$hhh$hh3Wa!!!aa$$aW!a!W!hhaW!!!WW$hWaa!Waah3aaaWWWW3h!aW;!!!hh!Wa$aW!hhWaa3WWW$$!W;haW!$h!!h&&!W3hWa+h$eW33!*hhheW3haa+3*W!$hWWa$W!W3$aWbh0a!3$WaW$!WW3ha!WWW!!h$a!WWaWa$3!!WWa!WhhWW!!W!a3h!aW!a!a3$3333h333$$hh3$$333$h$3h$$$a2!!Wa!d!dde:h2bggdf0:h2b12fdh3!33g1+$a$&3W$:Z$3:$+h$gZ&$$$3h1+Wfh3$3$&WW&3h$$h&Wa2h$f$$daZh3h333h2:3$+3Z$hg::3$3h$g2d*$Z12h@2d:3h$3h++Z3hg$:h32Z$$$h$$32a2hhf3$:aW1333h31aW&hh3$3&!Z1h33$$12:$hZ$g$3g+3W$&3!hZ1Z3h!$$+fZf0d$2bf2dd0:3gb+ga2!WW!!:W333h$$$$$eWaaa!WWbfhhhh3$311!WaaaWa&f:3hh3hg&fY!a!aWc1fcaWaWac&&cch$$YY&fYah$3!Yf&c:!WW2cff;d:!g2c&&Y:Y$Y2Y1fc:b&e+Yffcd*;e+Yf1;:*fe2;&fc:Y3c2Y1&Y:d!gg;f1;ZWW!2YffcW3h3aY1&c;3$3cYff;!!a!Wc1fc!!aWa;&fZh3h$3+&1WWaWWW!&&h$3h$hhfeW!WaaW!*h$$hh33h3;@&01b1b;$0dbW@+b3:d@af!X2+2*2Z;+d@Z*:hb30$ge*3h*c033e1$e03XX3&;3W;;c!$Y+ZWc1YagZ0a@3$h*a*!e:@$X+Xac$!03X!h;1Y1h1h1Y1!hg$!hZ3W1c&31$&c&;3a@hX!hY!0:e$X2*ae!0$h$*!X2:!c1;Wgdc$!Y;Y!3;&he@$Xb3f*33bce$$@Xd$*h0$2egb+:cg:@ZZ:0!&!Xgg$eZ*a02b$;e&@1b1bYW!3Wgh@X@3gd0g*ZdY*X!b1$Wa3dW!$Xh+03c*g&+Wb;bbY:Y@d@ZY;@Y*:!X3Y&22Y@Z11*h0*ceWe0X20@!WeZX@cea&:hdgb3&$1&b&+Wha$2bWZg;Z:WWh$&eYW0Yeecef1cb*;@dh*1*Yehdeg!*ed1WX+cc+:;Y32!*:agg!Zc@10@3@d;W+2b32:$dab*W*dX&;bf3e!a1+2WW!b+$chfgd@@&g@X10X&hY@:WfWgWY!", case1, case2),
            decrypt("!aaWWaWaWa!aWaaW!!!!aaaaaW!!!aW!WW!!WaWWa!WWWWaaaW!WW!WaWWW!!!!!!aa!aaW!!!a!YaaWaaWaa:!Waa!Wa!+!!!!a!!X:a!!!Waa$+faa!!!a:hheWWWW*3h$@W!WWW;h;!W!!aWXh0WWaWW!a$aW!a!!aa3!!W!aWaWW!!aW!W!aWaWaWWWW!aa!a!aaa!!W!aW!WWaW!!aW!aW!aa!WaaWaW!WaWWa!!WaWWWa$$$hh3$$hh$3h$h$3h$b3$$$h0$g:33h3$+dg1h$hh3f:$2!!h!aZ3hg:hc$2d$h+2b1e:d$321$1$&:h3gfhh$1:3$+1hh$1:3$3hh&33Z33$hh22Y:$h23$:+!:33hf+$h!dh3h:$d$+d3$2$3333d$3+f11f1Z3h+f2$dfd3$gg:Y2:d3h2d2Yd2d3h+!W3Wa:3+&$$h3h1:2d$h33$2:hbh3h$he3$$$3hh33hh$h3hhh3$$333$h$$h@!WW!!W!b&hh$$3hh&&W!aaWaa11:h$$3h+&fcaaWWWc&1;h$$3$Yf1Y0ehhXcf1Y0@h3X;&1Y$3*Y$Y1fc$f*3bc&&c3@+XX;&&cYcW03c1fcYd$;hY11c&hZ$h;f1c@d$3*Y&1cfbc:*Yf&Y$h2Z*c&1Y&$be3Y&&Y3&X@&YffY$hh$$Yff;!a!!!;1&d$333hgffWaa!aWa1fh$3hhh3f0aWa!aWW*3h$$$3$$h1hc333h33eaZa!W!aW13W:332&1e!:@!!a&&1hcX$;e!X1a*;*Yh3f0ff;f;aWf+2&c&c3g1aW&;*!Wa1$3f:;1$c&@!Y0;fac1f:;*Y&dg&1ab&;f!W*1Z0!;1:31&W:hcW!a*e3aa1eZh&2a:20a!WXYheW&d13&e0c+fa;aX20Yef$cf&0b!aWbcef&ghhh1;g&0!aW!Xc!&1h$3h@bgX1!aWWZfX10&Yg1!WWW3fY21$3$h$33$h3h$h$X@b10X@3+:+*30Zg:0bhY;Y$@*&dZ23dgg1$$XZhg@hh30hha33Xh&2+f3f:d&e$&h!3&$b0ch&11hYb@h&;1Yf$Xh*h&h1h0h3dYhh$Y+31h;b30c3&$Zchh$cg3hX3fh&h@30$&Yfc13*e;3&&f$;X0$f3W3f3b12+&3fZ:1$eh3a33X3h$b:3+0h3&Zd+3Zgg10X$;Yc$eb+:g*h0:+dhb0*&0Xe3$$h3hh$h3WW!b3eaW!a!a:W2a!a!aa2Ydaa!WWX@h*eaa!!Zbf02aaWa2dagZ!W!X@2Y:Xe!a:b@hX0gW!2d*&e+:W*0+ZWgd*eZXe2YZ*e2+Zbeh*e2d@2d*&egZ*0+dbe+Z*1&@2:W2db&e2dX30+:X+dXeh*@gddbe2W:X02be+Za2:be!gZ*h@gZa!:*ehX@+aab@+W:*eW!!2ZagdaWaWdX$@+Wa!Wb0$*@aa!WWg!:aW!!!W:32!a!Z:d:1+22+1cbWa;@cf2e0h$$**dXW*;fWa!e+W&chYf!:eWf;0aea*XYffhffYe0YWc0!&aX2YZ+;:+YZaY*afW!c@h&3!3!3fh*aeW0Wb!X++X23d0:d*a*!!;!cb+eWeYXab:X!ea@WbW*Z&+WYa:&2W;ba&WWce$!he$*$aheYac0!&ab*!0&$&*a@eW1Y@W0ab+!WYcca!ZXaXcf!W!@2e1hh3&*:f;X!!Y@c1d:dX&@ggg", case1, case2),
            decrypt("WWWWWWWW!aa!a!aWaW!aaa!WWaaa!W!aWa!aWa!a!WaWaW!!aW!!!!WaWaa!!WWW!W!aaaWWWWWaWW!WW!WaaZ!!!aWaWaaaa!a!!WXW@W!W!aWa!Xa!!W!aWXd@WWWWa@@X0aW!Wa0Xf!Wa!aW*ZaaWWaW!Wha!W!!!WW!!W!!!aaa!!aWWaW!W!aa!aW!!aaa!W!W!aWWWW!aa!aW!W!Waa!!aaa!W!!!!!a!!WWWWWa!a!!Wh:f$$31230!*h$3e!b0ch3$3$YXaa$h$3h!!&:h$h$$+1@Z$$$$3+*1hh3$3$hfb$hh3hhh0$h$hhh$$33$3$$hh3$$3$$eb$3$3333&f$33hh3$2Z$33h$3$2:$h33h3h&13$h$hhheb$h$33$3$hhh33$hh$$$3$hh$3h3$h$bhh$$$330f3h$h3$$1@Z$333h+bf:333$3+&Wa$h$33a!@Yh3hh$cb@a*3$3eab$:&333f+$!WWb$0!aa!a!da2a!!WWWe1X!W!!!Xeh*ea!!a;dh+Y!W!Wg;3;Zaaa**h3$00!aZWh&3!gWW@@cccX*a*@2:h+dXec:h@$b3g;g;201bZY:1$g$$3Z3feh1g1Zf$X&323$$Zh1+;2e1*:;dYZh0$b$gYX@+Zh+dXe!e0;YcbX!!:!3&$!+!ab*$3$0@!WWg;$cZ!!!WY:$gc!Wa!*ehX@a!W!!@&X!!!!!!ZagaWa!!abheaa!hh2W!ad$3$h*h$$0h$$+g!W!d:h$f*h1$@f33cgWgWZY$g&*3*:0&df;+X:@:Y1&;g!3W:cfcfb$h$@&cc&Xh3$e1ccfb3hh@&;Y0X$f$@*YYgXhXh&*;;g:$X:1!ccW$2eZ2;YY21h!3&dY;&b3330&ccfbhh3@fcYfb333@&Y1c+W;WdYffcg!gWZ;1+fbhX:0fd$;2*:e:;hh&*hW3@&3$g2!a!ZZh33bhh$0h$$3gaW!:$32!!@W*!WcX23e!bhd1f@YfWfY*dYd+h!$Z+fZ:a!aaWf1ce$Z1+h;fa2gX!0;+acf*hW$fcfZ&&@!*dZ1YXYd1+1!&3eZZa&@Z3!1c03;&cX*f22gghZZa*Wfbc*aWa!W*3YbWWab!f!+h:Zb&+$&2&cba1c0W*0Zh3@:3!31!f;*cb!0Zd1:&3ZfghY1cf!Waaagaa$+3!$d2fY@;&afYb1dg3e!X3:1YaW1a&!a:$2h$!$hdh*Za$c3W+@*cb:!+@c0hb$eYb3@hWehchch*ag;Zf&&2;:1Xg:&2defXf!$1h!f@*h*$&303ed+;W1WYdgfcg*&0Zc&21g:f2:1ZgZgha3:gda$3g;dh$!&0!WhaWb1a3hgcd3hW2Z2$!$Z+:+1g:&2:fZ&cg*f@ZY&Z2cWfa;ZgX3*hf$@3@*1Wh1$!&e1bgd&+Z012cdf&1g;da@hYhc$Xa3X30YX3@3Xc*da+@Ye*d!$c$a2@W!fYdXcXaWa*;ahb&dc1$b2gdYX!!dce1!c@d!03*W0;aY30hZ@a;WcbZ!aYZ23$Xaa@db2fa0b$@cfc2ZXc$;3ef!@XW*e$$@!2fYa!Z;;:aa2a&e$hWced3a:*$@*$&hg0&g$X0ahf!f$Xa!ch3bhg;XY:h$Y3ZeW1$hha;*!$feZc3a@aZbeg2:gY001h+X$gX$1f3eWZ@:$;X:Zcf;Y0X3YgWgZ!3h$Z&@:c!$b:eaZ!c3a", case1, case2),
            decrypt("!aaWWaWa!W!W!!a!aW!!WaaW!!WaWaWa!W!!WWW!!WaaW!WaW!WaaWW!!WaaWaa!!a!a!WW!aW!aaWa!!!!WeYba!WWa!;3YWaaW!Wb;eW!W!WW;!;aWaWaW3a3!Wa!a!YaY!!WaWWb;e!!a!!Wc$ca!WaWWe;*Wa!!a!WW!aaW!!WaaW!a!aWWWaa!W!WaWaWaWWaaWa!W!WW!aaWaaaaWaaa!aaWaa!W!a!!WW!WaW!!Wa!Wad!aaWaaW!dWa!WWWWWaaa!a!Wa!a!!a!a!aa!W!!W!a!!WW!aaWW!a!!!aWaWa!WWWaa!a!WaWWaaW*2aWa!WWWaa!!!!aW*2!0W!W!WX;!ea!WeWbdW@!a*;aWba!aaaYa!b+!W!!0WW!!WWa0cW!W!!a*X30aa!!a!he0WW!W!Wgc2aa!aZaZW@!WWWc@YZ!!a!WdaY!a!a!!!aYa!!aa!W!a!WW!W!a!WW!WWa!!aa!!WWWa$3$h$h$hhfg&h$$fdf21Z$$3gfd$;33h$hc3+1h$$3$1:fZ$$hh$2&23$$3hhhd&h3h3333&$3hb$2:333h3f$3d$$hh++2*$h33$h2$*h33h3hh;fh$3h33+1333$3$$&2h3$$$$he1b3$h$3heZ2d$hhh+333d$$$$gd$&33hfh$$h$$3f+$$h$h33Z1Zhh3h$gf+&$$h3$&:$Y$$hhh;$+&:3$hg1Z1+1h3$&:fhhh$$$hh3Wabh3$eWa;@&h$h1bYWW*h$h0!!;@WW!!!*YWa!W!WW!Waacc!ccaWcW!WWaW!;WXW!;W!e!3W*ca;ea33a!!!aaa$3X*c&c@e33aa!Wa!W$3aW;a;a!33aca;!;!h3!!Yaca!$3a!WaW!!hhXX;f;e03hWa!aW!W$3!bYac0!$a*WW;!!eacaWW!a!!Y!Wc;W;ca!aWaW!aWW!;@!aa!!*c!Wbh$$@WW;e&h$3&bYW!*h$3@WWh$$$$3hhheWWaW!!!*&3h$3hh3f&W!WWWWaf1:$$hhhg11;!WW!WYf&Yh$hh$c1&Y1@!*1Yf1Y3g&Z$;1&c3Z3+hY1f;121Z1Yf1ce&$1*Y&&;&h&$1;ff;e1af*;1&Y131h1Yff;0fhfb;&fc&+f:fc&1chd$+3YffY3g1d3;&&;10!Xf;&1c$$h$$Y&fYW!!a!c&1d$3hh$g11!!aaWWW1f$h3$3$3&0!W!W!Wa*hh3h3$h$$3233!33d3+!a&af!Wdf23@a*hZff0Y1Wf;b&fZ+3a$Zgfa:!W!W!+!1c$Zfg3Y1&2gX!0d:&1W&3W31!f3gb0!*@d$0c&d&+&;*+g$dW2$:d!!0Y$;bWaWaaW$aaWWaW@;hcbW!+g$:ag3:d0cfZfg&cX32b@W*edh&afh!$1a&f2+*a0Zd1fY3:fg3Y1WZa!aWa2afZ+$!$:2f&e;&!fYbffgh@!b3Z1gaWfW1aaZ$+hhW$h:hgd&:1g&+:@hXd!+@3**d+;&YZg0cba1&&a@c$a;2!:cW$3f21&&:fhfbbgfd@e1*d!e&*a+@+!13a$&WZa33@fX$$ahY&*a0fch2&*cWY01d1f+@ab:&&WWWaaa!aa&&+@WbZ1fg1X;W;0&Z3c1XWe1YhW$h@fbh$Wg!1$a3&WdbZa@f*!g@1bXg&:e01h&21f&:f3$acg!d;WhYb!&f&!@YXZgc1Yd20@$bZWg0$XgZfZf+&gZ", case1, case2),
            decrypt("$h3hhh$33h3h3hh3$$h33$3h$3$$h3h33$33$h3$$3333$h3hhh$h3hhh$$$$$$hh$h$$h$h3$3hhh$hhh$3333$hh$h3$33$3hh$$hh3h$hh3$hd$3$h3332b$$3$h$h$;hhh$3$$3@:$$h33hhghh3hh$$3hh3h$$3hh$$33$3hhhh3h3$3hh3hhhh3h3$3hh$33333$$3h$$$h$3$$$3h3$$hh$hhh$$$$3$3hhh333hh3$3WWaW!Wa!Wa!!!Waaaa!!!aaWaWWWW!WaaWWWaW!acaa!!!!WWhWaaaaaWW;a!W!aWa!3!!aa!W!*c0aWWa!WZhgWaW!Wa0hb!aaWWX03X0aaW*;$c$;@!adh$3$3gaaXc3chY@WaWXehb@a!!a!0hX!!!W!a:32WW!WWWX;ea!!a!aa$a!!WWa!W;W!aaa!W!$aW!!!aaaYW!!!!a!aW!aW!!aW!W!W!a!aW!a!!WWaWWaW!W!!3$h3h3hh33YhZ3+$Y323d131+3:fY3Y$;3c1$3+fh&Z$31c&h3$1c1g$:1Y1+hdh2&2$dfd3h1g1;fZf$fZ$Z323gfg&Y&Yf;&d3:&+3:1+h$$c1cfYh$h33hfh3$$3h;fY&chhhZfg3Zf+$2fc1;&;1Z&Z$dh23+13121Yf:f$h+f+3dfdh+3:1;f+3ZfY&h3h1c&$$g1$1Z$3fY$;3c3;&+3:&3f23d3;$:$ghchhhhh33$3h3hh$3hh3$@aWaaWW!*f33hhhh$&1aaa!WWW11:hh$3h+&&Ya!!WW;1fcha!bhYf1;@h33X;ffY1h@:bYf1;gaXZ:Y1fYY!@a;Y1fc;W*+Ycffc;$$Zgc1fYY3gcYY11c;3Z;cYf&c;aZg;c11Yd$h*;c11;$3W;YYf1c33Z*g;f&;$+1XeY1fc!a0ebY1&cW!Waac&fZ$$33hgf&!!a!!W!11h$$hhh3fe!aaa!WW*hhhh$hh33;;!WWWa;;1aW!!aWa1c$aWW!!3;bWe!W!bWe1a&h$hfWfXa1W!a&a@a3Y0$*;hWWa1YX;&Waadcgadc2aaY1;aa&YWa;;WhWc;aa;aY!aeY!W;:+3:2YaaYfY0!e;WWc+g;dZ;!W;W;WWeYWa;:WhW+YW!;1;W!fYWW:Ye&bc2WWW&Y*Y1WaahY0Y*;3W*!&WWW1ae1!133$1WfXa@!!WbWec3WW!!ah;&!!!!!!!&c;WWWaWYc0*WaaaW@X!03$$$hbaaYaW!aaYW;*:3h$+eYX*aaY!a000+d$fh+:*cW@!1aXW;c1cbWec1;c&X+!Z01cYf*0!b@1cc&*a!a@1cY&cW!!c1YY1eaa!X1cc2Wa!!adcY&eW!abfYc1;!!!Y&YY1*!!!@&YY1X0!*@fYY&X2!Z0&;Y&;*!eYfc;a@a1WbaY@+dh13+:*b*!!;WW@0;bdh$3g0c!YaW!WW;!!e$3hh$*!@*!a!Waebd:aZ;e!bb1aWWXaaXaZY*h@Zc*eg!*2bgYa!&2a@e!1YaZheXgaXYWg&ha@a!ad+hh!++!W13;Zg;df$2+Xb+gaa$f@a*Zhg1c;Zd00+*2d30aXbeYah+g;c@Wd@WZb;caaaW2$$WaZ:dZ$dh@;aa!$@:egeW*a!$f2a!deYf3hda+ddfXZ&Z@bhe:Wh33gZh3$$c3:+$aX13+X333$Z3&$!ah+gbhahea@Za;ZaY3@3321X230", case1, case2),
            decrypt("WWaWaW!ag!W!WWWaWa!aaaWWaWaaW!a!!Wa!!a!WWa!a!!WW!aa!a!aaaaa!W!a!!aWaWWXaaWa!W!!c!W!aaaWW!a!aaa!gW!!Wa!a!e!aW2!Wa!a!aWaWa!aaa!WWaWW!gW!W!aaaW!aaa!!!!Wa!WaaW!aa+*Ya!!!WW!aWW*Wb!aW*YWbW*!aaaaaaaaaWW!WWWaW!a!aWWWW!WaaWaWWaWa!aW!W!aaaWWWbWaW!aa!!bWW!!!W!WWaaW!aaWa!aa!aWW!!aWW!!!!WaWW!W!a!WWWW!aab;@Waaaa@3$$*W!W*ghhhde!W:3h3$3+!a*3$$$3e!!W3h3hh!WaX$$$3h0!!b$$$hheaa!:3$$+!!!ah$3$$WWaWdh3$2a!!!Z3$hg!aa!a1Yf!W!!!aaWWWaa!a!aaaa!a!WaaWWa!WWa!!!WaWW!WWWaaaaa!a!aaaa!WWWaW!aWa!a!!WaW!!W!a!!aa!Wa:h*hh3eh+e:X3$$02*@!$333$WX@fh$$3h1X+d3$hhh+dghh$3h3hdX$h$hh$3eb333$3hhehh$$hh3h$$$3@abhhhh3h1Zb33h3hh2*b$3$$$$;Zb33hh33ad*h$h$3$cYf3$$hhhbW&3$$3$h+e:$3hh$h$Y1hh$$33$e*$$3Xh3hhhh$@X$$h$$$h@ghh$$33hZ2:33$3h+de13$h3$&*@!33$h3WX@db$$30gbdhb3$h@3g$$hhhh$33h330Y*h3$+e$+$dh*:3*hf$13e$2:Z@hX+2ZghhX$e3$:$fh$Y3$&$h3+:f2Z$h3$e&&1b$h$+&hfh1:$1W2fh1:Wf@fh$ahh&*1$:gWd+h1$heYacX3h13Z2W:2h&e&33W3$1*fa21h1Za&hg131$1Z33301&f*$h3$+ZfgZh$3&3hch31$g3$b$03hZ+dZe3*g+Z$bh&hfh@hg0$+$d3bd3$3@Yb3$3$3h$33333XeXe3b0bed!Z*302W++W+:!2:WZ@*@+WZX@XaZb@W*egaW+ZaYa2:!*0gWhadX0Zb0b$eX@2gdaZ3gW2Z0gW3h3!:X*e*$3$@b@dWZ2a:2W+2!h2&:hW:2X$+!:$eZ2a$g&Z$!:dWZ2WZ+Wg*@*$3heXe0+a$$$!Z*2ZWZ$ga+ddX@bh@X0gX@2W3WZ*e!2dWc!g:aWdXeab@+a0*0+Wdbe*+!+da+d!d:!dX$@g!gb0be3beb@gWa@WXWacb+$f&&h:11@c1a&cX:;:2hW3:+f:ZaWWWa1fY03Zf+3Yf!+gX!eY2a;fbhW3&Y1:f&ea*:Z&;Xcd1gfa1heZZa1@d$W&ceh;&;*Zfggg2$d:;Xa1bcX!!a!WXhY*!!cXWfW+hZZd1+3&+1;b!1;0Wb0Z33eZh!31Wfc*cX!0dd&:f$Zf+hc&;&Waa!Wga!32hahdg&;@c1!1cb1Z+heWb3d&cWW&a&!a:$g33!h$d3g&1aeeg&fZ;32*Z3YW$df!Zhb3h030a1!f1@;Y2Wa*Z*d2!@adY&X0g2+2chcgW@aYZd!b!@f1Yg&&!!fh0cW+W$$!a+XaXW*:&h$YbXe@&hX;&3W@2ZgYZhYWf@bg2cb1hYg0$We1ba$Yccf22cW$0aW&bc$!@+2a+;X&Z$0$+3&*20e++bZ:3202$12Y&!3h@:a&&e!:d$!:YZa$da:*YZcgXe;&3h0:!a!@f&e1$2gZ0!XeX", case1, case2),
            decrypt("33$3$h$$h$aWWW!aa$+aaWWaaWZ0!!!Waaa*@!!WaaaaXe!WaWWa!*e!W!!W!aX@aWaWaaa*0aaW!!WWb0WaaWWaWb@a!!@e!!b@W!b3+!W*@WW!$eaWb@WaX$2aWX@W!aeeWa*@aa!a!!ab0aaWaaWWX0W!a!!a!X@Wa!WW!a*@!WWWaaaX@!aaaa!W*e!aW!!!Wb0W!!aWW!beaWWWa!abgaaW!W!a:3!WW!!!W$3$hh3hh$3*2:3hhg:@Y0X$$$0*;0Z3$h$$+b2$$hh333d&$h333$$fXh$h$$$$e*$hh$33he&h3h$$h313h3$$$:$33$3hh2:33$33h3$d$h$$h$3XZhhh$3hh@:3h$$h$he:hh$h33$h$$h$$$2hh:h3h3hgX2:$3$33db@Zh$33gW*WZ33&3$3$$$hf*3hhh$h3@*33h3$h3@133$$3h$&23$h$3$3:@Zh3$33+bc0*$$3eXcb+dh$32:@WaW!aW!!Wa!aaW!!!aaa!aaaW!W!a!WaWaW!!Z0!h!b2W!Y3hfh3;!!:2!!!:ga!b0;&;b@!abXf$f0ea!*XWW!00!W*cXW@ce!!bX;hce@WW::Xhe22W!YbXfe@YW!ddX3@22!WXbch;0eaWXY*!0;0aWXbW!!@0!a*b13feea!X0Y1;*0!WZ2aW!ZgWWc$$1h3c!W:0ah!XgWW!aa!Waaaaaa!!aaaa!aaWW!a!!W!!WWWWa!cgg3hh:Z;&b&@!X&01;e&aa!&*Y&ZeaWW*21e&Waa!!&bY+a!a!!:c3@Wa;a!X$@WaW3W!!X:!WXc@!!22a!dhgWWZ0!W1c&WW*@!X+$ZeWX@!Yf;&;!XeWh33$$Wb0!c&;&Y!*0W*+$Zeabea!fY1!ab+WW:$+WWZ:WaXceWW+0W!!haa!bh0a!c!Wb$;gaWa!!:;@&!!W!!&*fd0!WW*gfYe1aW!1bYfb1@!b&efYgg$3$ZZ;+1WaWWa1:@aea!!*aXaYYW!WY;We+bW1!0:b*1ea!Wbf0eYbc3;0YX;c;dW+ccYbaf+1df!@a&cbh0YfW!ac0cX;WaWX2;$cZ0W!!&a!!f!aaf:;aY+&W!*X2;d0eWW1:c!Y+f!aa&a!afa!!b+;3;d0!Wac0;*;WaW&;*3eY1WXW1+&:fW@c;c:a+;YY@YX;h;0c**&@!a!bfe0+bW&!@ZbaYY!!aYcWeaeaa!baXgf!aWa!1:f*W!aWW0fYe$3h3$XY1;a!a!a;1;&d3h$gf;bY!W;aW;0@fd$fh2&XYW0a1!X!cc1cX!e;fYc&*2&d@fc;1X@!b0&Y;&b;&;@&YY&Y*a0c&ccf&cfcffcY2!WWWWd;c1fc&Yff;;&;b!eYf;cfXYf;@fYcf*0ab@&Y;&b+f:e&;Y&;*!0Y1cc!@W&aX!;@1:3&3+&b*YWacaaceY1d$h$2&Y1;aWa!W;1Y0$h3$hb;&bWaWW!@f;@3$g*:ZZ1Whf+&*+*+!*dg!XfhZWX@*3$@ehZ$$Z@a*Y!:3hh@gWZ:dZ:he3bh@;Zfh0Y*Z22Y$3eW*@gd+Ze$$$1fYb&13hh*&be+c!WaX;b@+:*e&0eX02affhagX0+agaa!hh02;ah$3fgagZ$h31WaW2ZbWab0deaW1Z@a$f&e@&Z@g:2X@$$!ag@Wffb@3a$!aXf1ec@h:def1eW$c!$eehfX&3b@!gb@0+*b&&c333;X", case1, case2),
            decrypt("!Wd3$3+a!W!$3$$haaW*3333$eaWdh33hh2WW$h$3h$hab$h$3hh30:$3$3$hhgh$3h33hh3h3$$h3hhh$333$$hh3h$$3h333hhhhh3$3$$h333h$$33hh33$h3h3$h3hhh$3$3h33$33$$hh$hhh3$$$$3h3h$33hhhh$h3h33h33h3h3h:$33$h3h+bh3$3h330a3h$3$3$W!Z$hh3$gW!b3h33h0a!W$$$$$WaaWd$3h+W!!Y0;1;XYW!ba&c&W0W!!fc&Y1!!WW;f;fc!W0!f;1c&WXc!c1;fc!cf;&Yf;&Y&W1!0c*a&W*;@c1cX;0cfY1;fY1;f;&;&;fY1;1a1Y1!&Y&cfY1Y1;fY1Yf;&Y1Y&;f;fcfY&Y1!&;fWfY&Y1;1c&Y1Y1c&;1;&cX;@Y1Y*Y@!&!e;Xa&afYfcfcf;fYac1;&;WYe!fYf;fWXa!Y1Y1;W!W!&c&c1aW!*a1YfaeWWY@c1;XY!d3$h3$33g$aW!a!!!3g3h$hhhhZ&WaaaaWWf1WWW!Waa11!WWa!!!1&aaWWWWa&&W!WWWWa&&aWWWW!W&faWX$0!!&1aaYaYa!1&a!e;X!W1fW!@abaW&1W!fc1aWf&W!@WbaaffWW@YX!!ff!!YW;aa11!!bhe!a&1a!W!a!!f1a!aWW!W1&W!aaa!af&W!aaWa!ffW!!!aa!&faWaW!!Wf+hhhh$$3Z3aaWaaaahZ3hhh$3$+!a3hW$haaaXhbWe$0!a:+d!+:+aah@hh$*h!b$b3h$eh0ZgZ3$3+Zg$03$h33b$3*$h&hh@3+:3$Y$$g:033gh:$d**h$&&1W*@:3$:;$Wa@$$g+f;X$ef2f@c@bhaX$f@*hb$!a3+3eYX$03h$dgWa!e1hh$$e;b00h$0gghZXgdh$heZgZhb$af+Xe$h@$@fedXhdgd$e02:+b3*$+Xe$0!30$2ZX$!adgZ+gd2!!*3X3@h@aZf$:Y2h1g@&3:fg$f*@1$h&$h&*33@&hfX$$WfX+hd@1a$3eXh0*$$$1Y$$$Y&3$a3eYb3!$h;d:$2g;$b22dW+dZ@hcfe3X&c3$f&0hXf13Zhf:c213g0hhd&g$$XZ3fdc2fhg$1&e$*1fh$c&@$XfYh*2gZW2::03;:dh+2ch3W3@;b$W$31Y3$hY1$h$0b30*3$af*+3defW3he&3&b3$ef33133fb@&3:fg31b:f3:c2312h3$3$$3fheWa!!c0f*@$$h+!f&hfa@!@b0&X10:+@0b*Zf0XZYh&ZZ&@baYefW*021dg301&2&1+Z0!c&f+a*a$2X&g*3Z;a;;12eWWbh2+3f&:$2XW&f1Xca;ecd!1&:!f:e;XefW2faY:*+c3+W:c@1&g&ed!cZff!W+dWfYf&+e@a*fYXe*0da2bc*f1!W!!X:Xe&gdX;1Wf1&bWaZ03&0:$h$eZbb@0WWab@&X:hh3$h:3$g:Y1a21c;@eZ3*W3@f!f2h3&WWXWf;gc3!e+1Z!;Y&@;1Xah$chXfZc&*+2:&d!;+W2ZcWW@;:b&2&W0Z:0WYbbX3d+1Y:bZ3YX!d+;Wa031hWebXW:1Zc!dh&;*g$++YZWg0*f;@!0c0:+X0@:+b+W@0+bdbegcd!W2;!;gZh&cafX1&gdY0eb2;WfW@ddW&Wg@*&XbaechYYcbWY0+a0W2*WY31XY!Wg@d@!dacb1YYh;hb2d", case1, case2),
            decrypt("a!!!W!W!!!!W!!a!aW!WaaaW!WWaWaW!!!aaWa!aca!W!aW!Wh!aW!WaW!YaWa!Wa!ah!W!!Wa!XW@aaaWWaZW+Wa!W!ahf3!!WW!bgf:0aaaX;dag;e!W:ga!Wd+WWXY:!+Y0a!Wbgf:@WWaWah&h!WWW!!ZW2!aWa!!bWeWa!WW!ahaa!a!!aa;!WW!!aaWhWaWW!WaWcW!aaWa!aW!aW!W!aa!!!aWaW!!!WaW!aaWW!!WWW2$1&c1&hd3$$3h$h$hZ:0h1$*+2+$h$h$h$Zh+@hW3*Z$h@Z$h3g*3322&Y1:Z3c&$3h3h1c2&:f1fg&:33$3$$333:$Y$$3Yhg$hh$1h$33Z1d3!$212g1+g!:Z&Zd&:ha$2&g$h3h&$h$3:3Y33$Yh2$$h$hh$$$gf:1ff+&dc1$3$$3&ch2g1;fd:$$@dhhhgX3$gehW$bdh23h$3hh$d:d@h&hb++$h$h$3$$3g3f&cff3Z3$$3hh$hh0!a!!WaaX1$3hh3h$&&W!WaWa!1&dh$$h$211;aaa!aY11;$2WZhY1fc1hhh&;&f;$bW@$;&&;g1h&Zc1fc2Y$;:c&1;d13&2Y&&Yc$W$;Y&&;;$c3c;&f;Y$a$YY1fcdf$fgYffc2Y$YZcf1c+f$&Z;f&c$XWe$;&fYfhhhfY1f;3gadhcffYW!!!!Y1&:$3$3h2&1!W!!Waa&1h33h3$$&@!aa!!WWX33h3$h$h3@*$!Ya$0*!ZgWcWZ+aW$@Xa0X3a*$ad!g!$eZga2Ydadg$@Xe$X0*3$WZ!aW2W$+W2W$WZW:0*@*f0b0b!dWd!+!2!a2agYd!Zabe*&W&eXeaX:!a!+@!2:;Y;Yc2daX:!WW2@!*@X&afebeW2WgcZ!dW!Z!Z!+a+!@*eb1eb@*gW+!$aZ!Z3!:aaaga3$0b0$b@Xh:+W2;:!Z2X$ad!2a3eah@*W0X3Wad+!ca:+a@XhWY!30X&++2gWWY2@+$c@a!!@0g$h2!!W*0g2gZ!@aY0h0$2Xg*hh3+*!W0:31d&!!a!;03WX1ea!dab3$h0ac&a!2ZWebh!!Z0bh2XfX$g+aWgb&1e++a!a;e3:3&2!a$g3:haW0:hZ1::!a+dc!*Z*+WaXg!b2X@!aaW!!2d3eWWWa!+hde3h!aW$gb0+bWa!W0*+:ZW$@baX+Z+b03hWdX3@!0!haZ*Y@!$$+Wb*$eYa*bfhXb3$eb&33h$3$3$30$hhah3a:@Wagc!Zf1gZ+Wb0Yb*+3@+@3W@*2h*!;Z1@X2gWY$W!@:2@*2Z12!:+bZ3*!g:$$cZ3*a+h*hY$+*e$g*gabe3e3h*0;@X&a!@h&Xg*;a@a$@0h$1b!!:+gYZa&cXZgZWWWYWXd+:c!*&;:d2X:hca1!Z$@:ha&c@:$*adW!0*hf:@h@aYf30W*$!*W*h1eb1ahbhhfW*W2WX3hga$!3h$$h3$hh3333hWf++3:ZfWbbY1Y&;e0;$!hf$W3;1$Wb;0WhfbhXhfh0h@32d+1Z+:30!f;W;fWb2X*31h@@d!Zgfc1Z+WhZd:32++$f:2@WbZ+1:+:0YX2d21Zce&XY2f20*fY10bd1Z;ef*c21Z2de;b+:gfZgeWb:21$d:Zh2g+3Wd+f;&Z2W+bb$1h@@d@af;!Y1ab$+Z2&Z2:3bh*3&3@3@&habY@Wh&chahf$a3YX*;&YfceeW1ggh:d&a", case1, case2)
        ]

In [12]:
num_subjects = 40
sub_accepted_counter = 0
sub_rejected_counter = 0
num_patterns = 98
num_bins = 7
max_possible_clicks = int((grid_size / 3) * (grid_size / 3))
var_thresh = 80
openningside = 3

data2files = os.listdir("../../data/grid-search/pilot_data2/")
data3files = os.listdir("../../data/grid-search/pilot_data3/") + ["data_2022-04-27-13-43-42__2022-04-27-14-26-30_jkzy67vl.json"]

In [13]:
pattern_stats = pd.read_csv("../../csvs/grid-search/pattern_stats.csv")

In [15]:
IQR_clicks = []

info = pd.DataFrame(columns = ["pname", "pid", "timetaken", "gender", "age", "nationality", "hand", "device", "mouse", "strategy", "grids_most_opened", "grids_least_opened", "move_on", "gray", "OCD", "fun", "open_comments"])
grid_data = pd.DataFrame(columns = ["pname", "pid", "grid_id", "pattern_id", "num_clicks", "uLSC", "uInt", "fLSC_nondetrended", "fInt_nondetrended", "time_taken", "final_grid"])
click_data = pd.DataFrame(columns = ["pname", "pid", "grid_id", "pattern_id", "click_id", "move_on", "click_to_go", "uLSC", "uInt", "cLSC_nondetrended", "cInt_nondetrended", "time_taken", "positionx", "positiony", "current_grid"])
click_data_all = pd.DataFrame(columns = ["pname", "pid", "grid_id", "pattern_id", "click_id", "move_on", "click_to_go", "uLSC", "uInt", "cLSC_nondetrended", "cInt_nondetrended", "time_taken", "positionx", "positiony", "current_grid"])


info_rowid = 0
grid_data_rowid = 0
click_data_rowid = 0
click_data_all_rowid = 0

patternstr_to_patternid = dict(zip(pattern_stats["stimuli"], pattern_stats["pattern_id"]))

file_dir = "../../data/grid-search/"

# Takes 12 mins
for datafile in tqdm(sorted(os.listdir(file_dir))):
    if datafile[:4] != "data":
        continue
    
    print(datafile)
    
    if datafile in data2files:
        filedataset = 2
    elif datafile in data3files:
        filedataset = 3

    # if datafile == "data_2022-04-27-13-43-42__2022-04-27-14-10-23_jkzy67vl.json":       # the same file have been corrected and added. The name has changed so can ignore this old one now.
    #     continue

    if "jkzy67vl" in datafile:
        continue
    
    name = datafile.split('_')[-1][:-5]
    with open(file_dir + datafile) as json_file:
        data = json.load(json_file)['trials'][0]

    # get data
    exampleexplorationstimuli, order_of_ranges, order_within_ranges = get_exper_data(data)
    stimuli = get_pattern_order(order_of_ranges, order_within_ranges)
    time_taken, num_grids_visited, avg_clicks_per_grid, var_clicks_per_grid, clicks_per_grid, click_positions, time_per_grid, click_times, strategy, gridsmostopened, gridsleastopened, moveon, graytiles, ocd, fun, opencomment = get_behav_data(data, filedataset)
    IQR = np.percentile(clicks_per_grid, 75) - np.percentile(clicks_per_grid, 25)
    IQR_clicks.append(IQR)
    gender, age, nationality, dominant_hand, device, mouse, art_answers = get_demog_data(data)

    # if IQR == 0:
    #     continue

    if var_clicks_per_grid < 80:
        continue






    # write info
    info.loc[info_rowid] = [name, info_rowid + 1, time_taken, gender, age, nationality, dominant_hand, device, mouse, strategy, gridsmostopened, gridsleastopened, moveon, graytiles, ocd, fun, opencomment]

    # write grid_data
    for gid in range(num_grids_visited):
        pattern = stimuli[gid % num_patterns]
        pattern_id = patternstr_to_patternid[''.join(map(str, np.array(pattern).flatten()))]
        uLSC = float(pattern_stats[pattern_stats["pattern_id"] == pattern_id]["uLSC"])
        uInt = int(pattern_stats[pattern_stats["pattern_id"] == pattern_id]["uInt"])
        if uInt >= 300:      # Ignore outlier patterns (with int >= 300)
            continue

        grid = np.full((grid_size, grid_size), 2)

        # write click_data
        
        for cid in range(clicks_per_grid[gid]):
            grid = modify_grid(pattern, grid, click_positions[gid][cid])
            current_grid = ''.join(map(str, np.array(grid).flatten()))
            
            cLSC_nondetrended, cInt_nondetrended = calculate_measures(grid)
            click_data_all.loc[click_data_all_rowid] = [name, info_rowid + 1, gid + 1, pattern_id, cid + 1, int(cid + 1 == clicks_per_grid[gid]), clicks_per_grid[gid] - (cid + 1), uLSC, uInt, cLSC_nondetrended, cInt_nondetrended, click_times[gid][cid], click_positions[gid][cid][0], click_positions[gid][cid][1], current_grid]
            click_data_all_rowid += 1
            
            if clicks_per_grid[gid] != (grid_size//openningside) * (grid_size//openningside):       # only write data where num_clicks is NOT 81.
                click_data.loc[click_data_rowid] = [name, info_rowid + 1, gid + 1, pattern_id, cid + 1, int(cid + 1 == clicks_per_grid[gid]), clicks_per_grid[gid] - (cid + 1), uLSC, uInt, cLSC_nondetrended, cInt_nondetrended, click_times[gid][cid], click_positions[gid][cid][0], click_positions[gid][cid][1], current_grid]
                click_data_rowid += 1
            
        final_grid = ''.join(map(str, np.array(grid).flatten()))
        fLSC_nondetrended, fInt_nondetrended = calculate_measures(grid)
        grid_data.loc[grid_data_rowid] = [name, info_rowid + 1, gid + 1, pattern_id, clicks_per_grid[gid], uLSC, uInt, fLSC_nondetrended, fInt_nondetrended, time_per_grid[gid], final_grid]
        grid_data_rowid += 1

    info_rowid += 1

  0%|          | 0/44 [00:00<?, ?it/s]

data_2022-03-13-18-52-36__2022-03-13-19-15-41_jzptv7f8.json


  2%|▏         | 1/44 [00:33<24:06, 33.63s/it]

data_2022-03-13-19-00-47__2022-03-13-19-26-41_tg0ywfz0.json
data_2022-03-13-19-07-51__2022-03-13-19-29-25_ug8w8cl6.json


  7%|▋         | 3/44 [01:12<15:46, 23.09s/it]

...

data_2022-04-27-16-45-04__2022-04-27-17-20-55_0jvzsq15.json
data_2022-04-27-16-53-10__2022-04-27-17-21-10_ngrngpqz.json


 89%|████████▊ | 39/44 [10:00<00:54, 10.95s/it]

data_2022-04-27-16-55-03__2022-04-27-17-45-05_e3200kck.json


100%|██████████| 44/44 [10:19<00:00, 14.07s/it]

data_2022-04-27-17-02-44__2022-04-27-17-28-42_5y1buu2f.json


In [15]:
# info.to_csv("../../csvs/grid-search/info_reevaluated_forreproduction.csv", index = False)

In [16]:
# grid_data.to_csv("../../csvs/grid-search/grid_data_reevaluatedforreproduction.csv", index = False)

In [16]:
# click_data.to_csv("../../csvs/grid-search/click_data_reevaluatedforreproduction.csv", index = False)

In [ ]:
# click_data_all.to_csv("../../csvs/grid-search/click_data_all.csv", index = False)